# Introduction to Genki Widgets

A small plotting library which provides a python interface via jupyter widgets to the frontend application we are developing.

In [ ]:
import genki_widgets as gw

import os 
os.chdir('/Users/bjarnihaukurbjarnason/documents/genki-signals')

import genki_signals.signals as s
from genki_signals.buffers import DataBuffer
from genki_signals.data_sources import Sampler

In [ ]:
import time
from threading import Thread

from genki_signals.data_sources import SineWave

buffer = DataBuffer(max_size=400)
sine_wave = SineWave(frequency=1, amplitude=5, phase=0)
source = Sampler(sources={'sine': sine_wave}, sample_rate=100)

def run_source(buffer, source):
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 50)
        if not source.is_active:
            return
        
t = Thread(target=run_source, args=(buffer, source))
t.start()

In [ ]:
buffer

In [ ]:
# create a simple line plot
sig_x = {'key': 'timestamp', 'idx': 0}
sig_y = [{'key': 'sine', 'idx': 0}]
line = gw.Line(sig_x=sig_x, sig_y=sig_y, partial_options={'x_axis_visible': False, 'n_visible_points': 300})
line

In [ ]:
buffer.add_callback(line.update)

In [ ]:
# source.stop(); t.join()

In [ ]:
from genki_signals.data_sources import MouseDataSource


buffer = DataBuffer(max_size=100)
mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=1000)

t = Thread(target=run_source, args=(buffer, mouse_source))
t.start()

Each chart can display itself or it can be wrapped in a `gw.Dashboard` widget which enables us to have many charts in the same window.

In [ ]:
sig_x = {'key': 'timestamp', 'idx': 0}
sig_y = [
    {'key': 'mouse_position', 'idx': 0, 'name': 'y'},
    {'key': 'mouse_position', 'idx': 1, 'name': 'x'}
]

line = gw.Line(sig_x=sig_x, sig_y=sig_y, partial_options={'x_axis_visible': False})
bar = gw.Bar(sig_x=sig_x, sig_y=sig_y)

dashboard = gw.Dashboard(plots=[line, bar])
dashboard

In [ ]:
buffer.add_callback(dashboard.update)

This library can also handle an incredible throughput of data. A good example of which is the `gw.Spectrogram` widget.

In [ ]:
from genki_signals.system import System
from genki_signals.data_sources import MicDataSource

buffer = DataBuffer(400)
mic_source = MicDataSource()
derived = [
    s.FourierTransform('audio', name='spectro', window_size=2048, window_overlap=1024)
]

system = System(
    mic_source,
    derived
)

In [ ]:
import time
from threading import Thread

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 25)
        if not system.running: 
            return
        
t = Thread(target=run_system)
t.start()

In [ ]:
buffer

Here we create the aforementioned spectrogram which contains `1025x500=512.500` datapoints!

In [ ]:
sig_y = [{'key': 'spectro', 'idx': -1}]

spectro = gw.Spectrogram(
    sig_y=sig_y,
    partial_options = {
        'window_size': 2048,
        'sampling_rate': mic_source.sample_rate,
        'n_visible_windows': 500,
        'colormap_max': 3,
    }
)
spectro

In [ ]:
buffer.add_callback(spectro.update)


In [ ]:
1-1